In [9]:
!pip install beautifulsoup4
!pip install selenium

You should consider upgrading via the 'C:\Users\leedo\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\leedo\miniconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import re
import urllib.request
import os
import csv

In [2]:
url = "https://www.mangoplate.com/"
driver = webdriver.Chrome("./driver/chromedriver.exe")
driver.get(url)
driver.maximize_window()

C:\Users\leedo\AppData\Local\Temp\ipykernel_19824\3248022642.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver.exe")


In [4]:
# 검색창에 서울지역 검색
search_box = driver.find_element_by_xpath('//*[@id="main-search"]')
search_box.send_keys('서울')
search_box.send_keys(Keys.RETURN)

C:\Users\leedo\AppData\Local\Temp\ipykernel_19824\615670643.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_box = driver.find_element_by_xpath('//*[@id="main-search"]')


In [5]:
# 필터 선택
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
time.sleep(1)
# 지역 선택 서울-강남: 1, 서울-강북: 2
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[1]/a[{}]'.format(2)).click()

# 세부지역 선택 강남: 1~26, 강북: 1~25
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[2]/span[{}]/label/span'.format(4)).click()

# 지역 변수 저장 강남: 1~26, 강북: 1~25
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
district = soup.select("body > main > article > div.column-wrapper > div > aside > div.inner > div > div:nth-child(3) > p.metro > span:nth-child({}) > label > span".format(4))[0].text
district = district.replace("/", "_") + '_'

# 가격 만원미만, 1만원대, 2만원대, 3만원대 선택: 1 ~ 4
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(1)).click()

# 음식종류 선택: 한식, 일식, 중식, 양식, 세계음식, 뷔페, 카페, 주점: 1~8
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(1)).click()
time.sleep(1)

# 필터창 선택버튼
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

C:\Users\leedo\AppData\Local\Temp\ipykernel_19824\533304064.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_19824\533304064.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[1]/a[{}]'.format(2)).click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_19824\533304064.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[2]/span[{}]/label/span'.format(4)).click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_19824\533304064.py:17: DeprecationWarning:

In [229]:
detail = []
review_list = []

In [230]:
# total
# 리뷰만 크롤링
# 1 - 10 페이지, 2부터 시작
for s in range(1,9):
    try:
        for x in range(1,4):
            try:
                for z in range(2,11):
                    for q in range(1,11):
                        for w in range(1,3):
                            target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
                            target.send_keys(Keys.CONTROL +"\n")
                            time.sleep(0.3)
                            driver.switch_to.window(driver.window_handles[-1])
                            html = driver.page_source
                            soup = BeautifulSoup(html, "html.parser")

                            # 리뷰 개수
                            review_count_out = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > header > h2 > span.RestaurantReviewList__AllCount")[0].text

                            # 평점없으면 None
                            try:
                                point = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span")[0].text
                            except:
                                point = None


                            if int(review_count_out) >= 1:

                                # 더보기 칸까지 스크롤 내리기
                                try:
                                    driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
                                    action = ActionChains(driver)
                                    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                                except:
                                    pass
                                try:
                                    for e in range(int(review_count_out)//5):
                                        driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]').click()
                                        time.sleep(0.3)

                                        action = ActionChains(driver)
                                        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                                except:
                                    pass
                                html = driver.page_source
                                soup = BeautifulSoup(html, "html.parser")

                                # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                                review_count = len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1

                                # 실제 리뷰개수가 1개부터 조사
                                if review_count >= 1:
                                    # 가게 이름
                                    name = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1")[0].text
                                    save_path = "/Users/leedo/Documents/ds_study/" + district + name + "/"

                                    # 지역 + 가게 이름으로 폴더만들기
                #                     try:
                #                         if not os.path.isdir(save_path):
                #                             os.mkdir(save_path)
                #                     except:
                #                         pass

                                    driver.switch_to.window(driver.window_handles[-1])
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, "html.parser")

                                    detail_count = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr > th")

                                    for i in range(len(detail_count)):
                                        if detail_count[i].text == '주소':
                                            address = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text

                                            address = re.sub('\n', ' ', address).strip().split(' ')

                                            for i in range(len(address)):
                                                if address[i] == '지번':
                                                    a=address[1:i]
                                            address = ' '.join(a)


                                        elif detail_count[i].text == '음식 종류':

                                            category = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                                            category = re.sub('\n', ' ', category).strip().split(' ')
                                            category = category[3:]
                                            category = ' '.join(category)

                                        elif detail_count[i].text == '가격대':

                                            price = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                                            price = re.sub('\n', ' ', price).strip().split(' ')
                                            price = price[1:]
                                            price = ' '.join(price)


                                    # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                                    review_count = int(len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1)

                                    # 리뷰 크롤링
                                    try:
                                        for i in range(1,int(review_count)+1):
                                            tag = "body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li:nth-child({}) > a > div.RestaurantReviewItem__ReviewContent > div > p".format(i)
                                            review = soup.select(tag)
                                            review = review[0].text
                                            review = str(re.sub('\n', ' ', review).strip())

                                            # 맛있다 또는 괜찮다 표시
                                            recommend = driver.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[3]/span".format(i)).text
                                            if review != '':
                                                review_list.append({'가게이름':name, '리뷰':review, '맛평가': recommend})
                                    except:
                                        pass

                                    detail.append({'가게이름':name,'가게 주소':address,'음식 종류':category,'가격 범위':price,"리뷰개수":review_count,'평점':point})

                                    # 리뷰 펼친 후 스크롤이 해당 지점에 못 미쳐서 클릭 조회가 안되던 걸 리뷰 개수 위치로 스크롤 이동 후 해결
                                    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/header/h2/span[4]')).perform()
                                    # n 번째 리뷰 클릭후 썸네일 다운
                                    count = 0

                                    try:               
                #                         for r in range(review_count):
                #                             tag = '/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[2]/div/p'.format(r+1)
                #                             driver.find_element_by_xpath(tag).click()

                #                             action = ActionChains(driver)
                #                             action.move_to_element(driver.find_element_by_xpath(tag)).perform()

                #                             time.sleep(0.5)

                #                             driver.switch_to.window(driver.window_handles[-1])
                #                             html = driver.page_source
                #                             soup = BeautifulSoup(html, "html.parser")

                #                             # 썸네일 갯수
                #                             html = driver.page_source
                #                             soup = BeautifulSoup(html, "html.parser")
                #                             if len(soup.select("body > main > section > div > div")) != 0:
                #                                 picture_count = int((len(soup.select("body > main > section > div > div")[0]) - 1) / 2)

                #                                 # n 번째 썸네일 url확인 후 저장
                #                                 for t in range(picture_count):
                #                                     count += 1
                #                                     try:
                #                                         image_url = driver.find_element_by_xpath("/html/body/main/section/div/div/div[{}]".format(t+1)).get_attribute("style")
                #                                         a = image_url.index('(')
                #                                         b = image_url.index(')')
                #                                         image_url = image_url[a+2:b-1]
                #                                         urllib.request.urlretrieve(image_url, save_path +'{}.jpg'.format(count))
                #                                         time.sleep(0.1)
                #                                     except:
                #                                         pass
                #                                 driver.close()  #링크 이동 후 탭 닫기
                #                                 driver.switch_to.window(driver.window_handles[-1])
                #                                 time.sleep(0.1)
                #                             else:
                #                                 driver.close()  #링크 이동 후 탭 닫기
                #                                 driver.switch_to.window(driver.window_handles[-1])
                #                                 time.sleep(0.1)

                                        driver.close()  #링크 이동 후 탭 닫기
                                        driver.switch_to.window(driver.window_handles[-1])
                                        time.sleep(0.1)
                                    except:
                                        driver.close()  #링크 이동 후 탭 닫기
                                        driver.switch_to.window(driver.window_handles[-1])
                                        time.sleep(0.1)

                                else:            
                                    driver.close()  #링크 이동 후 탭 닫기
                                    driver.switch_to.window(driver.window_handles[-1])
                                    time.sleep(0.1)
                            else:
                                driver.close()  #링크 이동 후 탭 닫기
                                driver.switch_to.window(driver.window_handles[-1])
                                time.sleep(0.1)
                    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{}]'.format(z)).click()
                    time.sleep(1)
            except:
                pass
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 필터선택 버튼 스크롤
            action = ActionChains(driver)
            action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]')).perform()

            # 필터 선택
            driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
            time.sleep(0.1)

            # 가격선택 버튼 스크롤
            action = ActionChains(driver)
            action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[1]')).perform()
            time.sleep(0.1)

            # 가격 만원미만, 1만원대, 2만원대, 3만원대 선택: 1 ~ 4
            driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x)).click()
            driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x+1)).click()
            time.sleep(0.1)

            # 필터창 선택버튼
            driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
            time.sleep(0.1)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

        # 필터선택 버튼 스크롤
        action = ActionChains(driver)
        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]')).perform()    

        # 필터 선택
        driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
        time.sleep(0.1)

        driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[1]').click()
        driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x+1)).click()

        # 음식종류선택 버튼 스크롤
        action = ActionChains(driver)
        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[1]')).perform()

        # 음식종류 선택: 한식, 일식, 중식, 양식, 세계음식, 뷔페, 카페, 주점: 1~8
        driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(s)).click()
        driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(s+1)).click()
        time.sleep(0.1)

        # 필터창 선택버튼
        driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
        time.sleep(0.1)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
    except:
        pass
    

C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\2112093229.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\2112093229.py:32: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\2112093229.py:34: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\2112093229.py:39: Deprecatio

In [205]:
# 리뷰만 크롤링
# 1 - 10 페이지, 2부터 시작
for z in range(2,11):
    for q in range(1,11):
        for w in range(1,3):
            target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
            target.send_keys(Keys.CONTROL +"\n")
            time.sleep(0.3)
            driver.switch_to.window(driver.window_handles[-1])
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 리뷰 개수
            review_count_out = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > header > h2 > span.RestaurantReviewList__AllCount")[0].text

            # 평점없으면 None
            try:
                point = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span")[0].text
            except:
                point = None


            if int(review_count_out) >= 1:

                # 더보기 칸까지 스크롤 내리기
                try:
                    driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
                    action = ActionChains(driver)
                    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                except:
                    pass
                try:
                    for e in range(int(review_count_out)//5):
                        driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]').click()
                        time.sleep(0.3)

                        action = ActionChains(driver)
                        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                except:
                    pass
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")

                # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                review_count = len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1

                # 실제 리뷰개수가 1개부터 조사
                if review_count >= 1:
                    # 가게 이름
                    name = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1")[0].text
                    save_path = "/Users/leedo/Documents/ds_study/" + district + name + "/"

                    # 지역 + 가게 이름으로 폴더만들기
#                     try:
#                         if not os.path.isdir(save_path):
#                             os.mkdir(save_path)
#                     except:
#                         pass

                    driver.switch_to.window(driver.window_handles[-1])
                    html = driver.page_source
                    soup = BeautifulSoup(html, "html.parser")

                    detail_count = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr > th")

                    for i in range(len(detail_count)):
                        if detail_count[i].text == '주소':
                            address = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text

                            address = re.sub('\n', ' ', address).strip().split(' ')

                            for i in range(len(address)):
                                if address[i] == '지번':
                                    a=address[1:i]
                            address = ' '.join(a)


                        elif detail_count[i].text == '음식 종류':

                            category = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                            category = re.sub('\n', ' ', category).strip().split(' ')
                            category = category[3:]
                            category = ' '.join(category)

                        elif detail_count[i].text == '가격대':

                            price = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                            price = re.sub('\n', ' ', price).strip().split(' ')
                            price = price[1:]
                            price = ' '.join(price)


                    # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                    review_count = int(len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1)

                    # 리뷰 크롤링
                    try:
                        for i in range(1,int(review_count)+1):
                            tag = "body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li:nth-child({}) > a > div.RestaurantReviewItem__ReviewContent > div > p".format(i)
                            review = soup.select(tag)
                            review = review[0].text
                            review = str(re.sub('\n', ' ', review).strip())
                            
                            # 맛있다 또는 괜찮다 표시
                            recommend = driver.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[3]/span".format(i)).text
                            if review != '':
                                review_list.append({'가게이름':name, '리뷰':review, '맛평가': recommend})
                    except:
                        pass

                    detail.append({'가게이름':name,'가게 주소':address,'음식 종류':category,'가격 범위':price,"리뷰개수":review_count,'평점':point})

                    # 리뷰 펼친 후 스크롤이 해당 지점에 못 미쳐서 클릭 조회가 안되던 걸 리뷰 개수 위치로 스크롤 이동 후 해결
                    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/header/h2/span[4]')).perform()
                    # n 번째 리뷰 클릭후 썸네일 다운
                    count = 0

                    try:               
#                         for r in range(review_count):
#                             tag = '/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[2]/div/p'.format(r+1)
#                             driver.find_element_by_xpath(tag).click()

#                             action = ActionChains(driver)
#                             action.move_to_element(driver.find_element_by_xpath(tag)).perform()

#                             time.sleep(0.5)

#                             driver.switch_to.window(driver.window_handles[-1])
#                             html = driver.page_source
#                             soup = BeautifulSoup(html, "html.parser")

#                             # 썸네일 갯수
#                             html = driver.page_source
#                             soup = BeautifulSoup(html, "html.parser")
#                             if len(soup.select("body > main > section > div > div")) != 0:
#                                 picture_count = int((len(soup.select("body > main > section > div > div")[0]) - 1) / 2)

#                                 # n 번째 썸네일 url확인 후 저장
#                                 for t in range(picture_count):
#                                     count += 1
#                                     try:
#                                         image_url = driver.find_element_by_xpath("/html/body/main/section/div/div/div[{}]".format(t+1)).get_attribute("style")
#                                         a = image_url.index('(')
#                                         b = image_url.index(')')
#                                         image_url = image_url[a+2:b-1]
#                                         urllib.request.urlretrieve(image_url, save_path +'{}.jpg'.format(count))
#                                         time.sleep(0.1)
#                                     except:
#                                         pass
#                                 driver.close()  #링크 이동 후 탭 닫기
#                                 driver.switch_to.window(driver.window_handles[-1])
#                                 time.sleep(0.1)
#                             else:
#                                 driver.close()  #링크 이동 후 탭 닫기
#                                 driver.switch_to.window(driver.window_handles[-1])
#                                 time.sleep(0.1)

                        driver.close()  #링크 이동 후 탭 닫기
                        driver.switch_to.window(driver.window_handles[-1])
                        time.sleep(0.1)
                    except:
                        driver.close()  #링크 이동 후 탭 닫기
                        driver.switch_to.window(driver.window_handles[-1])
                        time.sleep(0.1)

                else:            
                    driver.close()  #링크 이동 후 탭 닫기
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(0.1)
            else:
                driver.close()  #링크 이동 후 탭 닫기
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(0.1)
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{}]'.format(z)).click()
    time.sleep(1)

C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\4282595061.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\4282595061.py:27: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\4282595061.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
C:\Users\leedo\AppData\Local\Temp\ipykernel_22636\4282595061.py:34: Deprecation

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[7]/div[2]/figure/figcaption/div/a"}
  (Session info: chrome=103.0.5060.53)
Stacktrace:
Backtrace:
	Ordinal0 [0x005D56B3+2184883]
	Ordinal0 [0x0056E5F1+1762801]
	Ordinal0 [0x00483DA8+802216]
	Ordinal0 [0x004B1B40+990016]
	Ordinal0 [0x004B1DDB+990683]
	Ordinal0 [0x004DEC32+1174578]
	Ordinal0 [0x004CCB54+1100628]
	Ordinal0 [0x004DCF52+1167186]
	Ordinal0 [0x004CC926+1100070]
	Ordinal0 [0x004A6EA0+945824]
	Ordinal0 [0x004A7D96+949654]
	GetHandleVerifier [0x00874192+2704034]
	GetHandleVerifier [0x0086687D+2648461]
	GetHandleVerifier [0x0066119A+529066]
	GetHandleVerifier [0x00660006+524566]
	Ordinal0 [0x00574F2B+1789739]
	Ordinal0 [0x00579978+1808760]
	Ordinal0 [0x00579A65+1808997]
	Ordinal0 [0x00582A01+1845761]
	BaseThreadInitThunk [0x7762FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77837A9E+286]
	RtlGetAppContainerNamedObjec

In [323]:
driver.switch_to.window(driver.window_handles[-1])
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [231]:
detail

[{'가게이름': '혜화도담',
  '가게 주소': '서울특별시 종로구 대명1길 16-6',
  '음식 종류': '한정식 / 백반 / 정통 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 26,
  '평점': '4.3'},
 {'가게이름': '청산손만두',
  '가게 주소': '서울특별시 종로구 대학로11길 19-6',
  '음식 종류': '탕 / 찌개 / 전골',
  '가격 범위': '만원 미만',
  '리뷰개수': 10,
  '평점': '4.3'},
 {'가게이름': '혜화칼국수',
  '가게 주소': '서울특별시 종로구 창경궁로35길 13',
  '음식 종류': '국수 / 면 요리',
  '가격 범위': '만원 미만',
  '리뷰개수': 56,
  '평점': '4.2'},
 {'가게이름': '나누미떡볶이',
  '가게 주소': '서울특별시 종로구 성균관로 9-1',
  '음식 종류': '기타 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 5,
  '평점': '4.1'},
 {'가게이름': '왕십리순대국',
  '가게 주소': '서울특별시 종로구 율곡로 220-1',
  '음식 종류': '탕 / 찌개 / 전골',
  '가격 범위': '만원 미만',
  '리뷰개수': 7,
  '평점': '4.0'},
 {'가게이름': '봉쥬떡볶이',
  '가게 주소': '서울특별시 종로구 대명길 4 B1',
  '음식 종류': '기타 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 8,
  '평점': '3.9'},
 {'가게이름': '명륜 왕만두',
  '가게 주소': '서울특별시 종로구 성균관로 4-2',
  '음식 종류': '한정식 / 백반 / 정통 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 10,
  '평점': '3.8'},
 {'가게이름': '순대실록',
  '가게 주소': '서울특별시 종로구 동숭길 127 우성빌딩',
  '음식 종류': '탕 / 찌개 / 전골',
  '가격 범위': '만원 미만',
 

In [232]:
review_list

[{'가게이름': '혜화도담', '리뷰': '가성비, 런치에 가고싶다. 분위기도 오짐. 친구들과 함께', '맛평가': '맛있다'},
 {'가게이름': '혜화도담',
  '리뷰': '2월 11일 방문, 1시 반쯤 갔을 때는 사람이 많이 빠진 상태라 바로 앉아서 먹을 수 있었다.점심메뉴 김치찜(8.0) 2인분과 대파전(12.0) 하나를 시켰는데 너무 많이 시켰다 싶었다. 대파전은 매우 바삭바삭하면서도 대파향이 굉장해서 맛있었고 김치찜은 고기도 부드럽고 김치도 맛있었다.재방문의사 있긴한데 다른 혜화 맛집 때문에 밀릴 듯.',
  '맛평가': '맛있다'},
 {'가게이름': '혜화도담',
  '리뷰': '이 근처 학교 다니던 친구가 본인은 과제하다 열받으면 여기서 막걸리를 퍼마신다길래 저장해놨던 곳인데 드디어 방문. 주말 저녁 5시 조금 넘어서 방문했는데도 사람이 엄청 많았고 운좋게 착석할 수 있었다. 그리고 이른 시간임에도 불구하고 직접 제조했다는 막걸리는 거의 품절이고 한 종류만 가능하길래 주문. 전과 김치찜을 주문했는데 우선 전은 안까지 바삭바삭한 것이 독특했고 비법을 좀 듣고 싶었다... 아주 도톰해서 그런지 채소가 그대로 느껴지는 듯했고 마치 분식집 야채튀김을 먹는 느낌...그리고 두꺼운 만큼 좀 물리기도 해서 김치찜 같이 완화해줄 수 있는 메뉴 동반 주문 필수. 김치찜은 살짝 달달한데 고기는 보들보들하고 잡내가 없다. 김치는 아삭하기도 하지만 엄청 부드럽기도 하다. 술이 쭉쭉 들어감... 쌀밥 좋아하는 사람들은 꼭 밥이랑 같이 먹어야 할 것 같다. 술마시는 곳이라 조금 북적북적하긴 한데 너무 힘든 정도는 아님.',
  '맛평가': '맛있다'},
 {'가게이름': '혜화도담',
  '리뷰': '⭐️ 한줄평: 김치찜 최고*\\(^o^)/*1. 위치 및 분위기맛집이 많은 대학로 골목에 있는 또 하나의 맛집. 위층도 유명한 집이고 옆에 있는 독일주택도 웨이팅이 있는 곳으로 알고있는데 그 사이에 위치해있다. 분위기는 그냥 깔끔한 한식집 느낌. 테이블 사이의 간격이 

In [233]:
review_frame = pd.DataFrame(review_list)
review_frame

,가게이름,리뷰,맛평가
0,혜화도담,"가성비, 런치에 가고싶다. 분위기도 오짐. 친구들과 함께",맛있다
1,혜화도담,"2월 11일 방문, 1시 반쯤 갔을 때는 사람이 많이 빠진 상태라 바로 앉아서 먹을...",맛있다
2,혜화도담,이 근처 학교 다니던 친구가 본인은 과제하다 열받으면 여기서 막걸리를 퍼마신다길래 ...,맛있다
3,혜화도담,⭐️ 한줄평: 김치찜 최고*\(^o^)/*1. 위치 및 분위기맛집이 많은 대학로 골...,맛있다
4,혜화도담,김치찜 달지 않고 괜찮았는데 고기가 좀만 더 많았으면 해요..!,맛있다
...,...,...,...
3826,야스미,아나고텐동튀김이아주많다좀 느끼3.6/5,맛있다
3827,달빛마루,좋아요,맛있다
3828,천지주점,쫀좋 추천세트로 시키면 양이 푸짐자리가 조금 비좁고 비흡연석이 없다는 거,맛있다
3829,소호17번지,아란 매크리 무어 피티드옥토모어 스코티시 발리옥토모어 아일라 발리 마셨습니다.사장님...,맛있다


In [234]:
detail_frame = pd.DataFrame(detail)
detail_frame

,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점
0,혜화도담,서울특별시 종로구 대명1길 16-6,한정식 / 백반 / 정통 한식,만원 미만,26,4.3
1,청산손만두,서울특별시 종로구 대학로11길 19-6,탕 / 찌개 / 전골,만원 미만,10,4.3
2,혜화칼국수,서울특별시 종로구 창경궁로35길 13,국수 / 면 요리,만원 미만,56,4.2
3,나누미떡볶이,서울특별시 종로구 성균관로 9-1,기타 한식,만원 미만,5,4.1
4,왕십리순대국,서울특별시 종로구 율곡로 220-1,탕 / 찌개 / 전골,만원 미만,7,4.0
...,...,...,...,...,...,...
493,야스미,서울특별시 종로구 대명길 39-10,일반 주점,2만원-3만원,2,None
494,달빛마루,서울특별시 종로구 대학로11길 24-1,일반 주점,2만원-3만원,1,None
495,천지주점,서울특별시 종로구 대학로11길 34-1 B1,전통 주점 / 포차,2만원-3만원,1,None
496,소호17번지,서울특별시 종로구 동숭3길 16,칵테일 / 와인,2만원-3만원,1,None


In [210]:
district

'대학로_혜화_'

In [235]:
review_frame.to_csv('C:/Users/leedo/Documents/ds_study/{}_review_list.csv'.format(district + 'total'), encoding='utf-8-sig')

In [236]:
detail_frame.to_csv('C:/Users/leedo/Documents/ds_study/{}_detail.csv'.format(district + 'total'), encoding='utf-8-sig')